In [0]:
# Upgrade Databricks SDK to the latest version and restart Python to see updated packages
%pip install --upgrade databricks-sdk==0.49.0
%restart_python

In [0]:
from databricks.sdk.service.jobs import JobSettings as Job

In [0]:
medalion_pipeline = Job.from_dict(
    {
        "name": "medalion_pipeline",
        "description": "Pipeline de dados referente a processos de arquitetura medalion do projeto nintendo",
        "email_notifications": {
            "on_failure": [
                "felipegoraro@outlook.com",
            ],
            "on_duration_warning_threshold_exceeded": [
                "felipegoraro@outlook.com",
            ],
        },
        "timeout_seconds": 570,
        "health": {
            "rules": [
                {
                    "metric": "RUN_DURATION_SECONDS",
                    "op": "GREATER_THAN",
                    "value": 540,
                },
            ],
        },
        "trigger": {
            "pause_status": "UNPAUSED",
            "file_arrival": {
                "url": "abfss://nintendo@nintendostorageaccount.dfs.core.windows.net/inbound/",
            },
        },
        "tasks": [
            {
                "task_key": "bronze",
                "notebook_task": {
                    "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/medalion/bronze",
                    "source": "WORKSPACE",
                },
                "job_cluster_key": "job_nintendo",
            },
            {
                "task_key": "silver",
                "depends_on": [
                    {
                        "task_key": "bronze",
                    },
                ],
                "notebook_task": {
                    "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/medalion/silver",
                    "source": "WORKSPACE",
                },
                "job_cluster_key": "job_nintendo",
            },
            {
                "task_key": "gold",
                "depends_on": [
                    {
                        "task_key": "silver",
                    },
                ],
                "notebook_task": {
                    "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/medalion/gold",
                    "source": "WORKSPACE",
                },
                "job_cluster_key": "job_nintendo",
            },
        ],
        "job_clusters": [
            {
                "job_cluster_key": "job_nintendo",
                "new_cluster": {
                    "cluster_name": "",
                    "spark_version": "15.4.x-scala2.12",
                    "node_type_id": "Standard_D4s_v3",
                    "spark_env_vars": {
                        "PYSPARK_PYTHON": "/databricks/python3/bin/python3",
                    },
                    "data_security_mode": "DATA_SECURITY_MODE_DEDICATED",
                    "runtime_engine": "STANDARD",
                    "kind": "CLASSIC_PREVIEW",
                    "is_single_node": True,
                },
            },
        ],
        "tags": {
            "project": "nintendo",
            "type": "medalion",
        },
        "queue": {
            "enabled": True,
        },
    }
)

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
w.jobs.reset(new_settings=medalion_pipeline, job_id=1013005912257396)
# or create a new job using: w.jobs.create(**medalion_pipeline.as_shallow_dict())


In [0]:
modeling_pipeline = Job.from_dict(
    {
        "name": "modeling_pipeline",
        "description": "Pipeline de dados referente a processos de modelagem das tabelas do projeto nintendo em fato/dimensão",
        "email_notifications": {
            "on_failure": [
                "felipegoraro@outlook.com",
            ],
            "on_duration_warning_threshold_exceeded": [
                "felipegoraro@outlook.com",
            ],
        },
        "timeout_seconds": 570,
        "health": {
            "rules": [
                {
                    "metric": "RUN_DURATION_SECONDS",
                    "op": "GREATER_THAN",
                    "value": 540,
                },
            ],
        },
        "trigger": {
            "pause_status": "UNPAUSED",
            "file_arrival": {
                "url": "abfss://nintendo@nintendostorageaccount.dfs.core.windows.net/gold/",
            },
        },
        "tasks": [
            {
                "task_key": "table-details",
                "notebook_task": {
                    "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/normalize/tabelas-detalhes",
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "table-price",
                "notebook_task": {
                    "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/normalize/tabelas-preço",
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "table-products",
                "notebook_task": {
                    "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/medalion/bronze",
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "oled-metric",
                "depends_on": [
                    {
                        "task_key": "table-products",
                    },
                    {
                        "task_key": "table-details",
                    },
                    {
                        "task_key": "table-price",
                    },
                ],
                "notebook_task": {
                    "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/metrics/best-price-oled",
                    "source": "WORKSPACE",
                },
            },
        ],
        "job_clusters": [
            {
                "job_cluster_key": "job_nintendo",
                "new_cluster": {
                    "cluster_name": "",
                    "spark_version": "15.4.x-scala2.12",
                    "node_type_id": "Standard_D4s_v3",
                    "spark_env_vars": {
                        "PYSPARK_PYTHON": "/databricks/python3/bin/python3",
                    },
                    "data_security_mode": "DATA_SECURITY_MODE_DEDICATED",
                    "runtime_engine": "STANDARD",
                    "kind": "CLASSIC_PREVIEW",
                    "is_single_node": True,
                },
            },
        ],
        "tags": {
            "project": "nintendo",
            "type": "dimensional modeling",
        },
        "queue": {
            "enabled": True,
        },
        "performance_target": "STANDARD",
    }
)

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
w.jobs.reset(new_settings=modeling_pipeline, job_id=23099207674253)
# or create a new job using: w.jobs.create(**modeling_pipeline.as_shallow_dict())
